### Perform Drude-Smith fit on experimental data

In [ ]:
import os
from drude_smith_fit import check_input_parameters
from drude_smith_fit import read_csv
from drude_smith_fit import perform_fit
from drude_smith_fit import plot_experimental_and_fitted_data
from drude_smith_fit import print_fit_results
from drude_smith_fit import write_csv, write_parameters
from drude_smith_fit import guess_min_Lorentz_f

Define input

In [ ]:
# Specify directory containing experimental datasets in csv files
input_directory = "data"

min_frequency = 0.3E12
max_frequency = 2.2E12

Define directory for output

In [ ]:
output_directory = "output"  # Must already exist

Fix parameters

In [ ]:
fix_phi = 1.
fix_m = False
fix_tau = False  # fix tau in fs
fix_c1 = False
fix_c2 = 0.
fix_c3 = 0.
fix_phi_ex = 0.
fix_fbn = 0.
fix_wbn = 0.  # fix wbn in THz
fix_gamma = 0.  # fix gamma in THz

Perform loop over input files

In [ ]:
for filename in os.listdir(input_directory):
    if filename.endswith(".csv"):
        input_filename = os.path.join(input_directory, filename)

        base_name = os.path.splitext(os.path.basename(input_filename))[0]
        image_filename = os.path.join(output_directory, base_name + '.png')
        param_filename = os.path.join(output_directory, base_name + '_param.csv')
        data_filename = os.path.join(output_directory, base_name + '_fitted.csv')

        frequencies, complex_numbers = read_csv(
            input_filename, min_frequency, max_frequency
        )
        # Convert to m^2 V^-1 s^-1.
        complex_numbers *= 1.e-4
        
        min_Lorentz_f = guess_min_Lorentz_f(frequencies, complex_numbers.real)
        num_variable_params = check_input_parameters(
            fix_phi, fix_m, fix_tau, fix_c1, fix_c2, fix_c3,
            fix_phi_ex, fix_fbn, fix_wbn, fix_gamma, min_Lorentz_f
        )

        try:
            fitted_complex_numbers, \
                [phi_fit, m_fit, tau_fit, c1_fit, c2_fit, c3_fit,
                phi_ex_fit, fbn_fit, wbn_fit, gamma_fit], \
                std_dev = perform_fit(
                    frequencies, complex_numbers, num_variable_params
                )

            print("Results for ", input_filename)

            print_fit_results(
                phi_fit, m_fit, tau_fit, c1_fit, c2_fit, c3_fit,
                phi_ex_fit, fbn_fit, wbn_fit, gamma_fit, std_dev
            )

            title = ("phi = %.3e, m = %.3e, tau = %.3e,\n"
                    "c1 = %.3e, c2 = %.3e, c3 = %.3e,\n"
                    "phi_ex = %.3e, fbn = %.3e, wbn = %.3e, gamma = %.3e")
            plot_experimental_and_fitted_data(
                frequencies, complex_numbers, fitted_complex_numbers,
                title
                % (phi_fit, m_fit, tau_fit, c1_fit, c2_fit, c3_fit,
                phi_ex_fit, fbn_fit, wbn_fit, gamma_fit),
                image_filename
            )

            write_parameters(
                param_filename, phi_fit, m_fit, tau_fit,
                c1_fit, c2_fit, c3_fit, phi_ex_fit, fbn_fit, wbn_fit, gamma_fit, std_dev
            )

            write_csv(
                data_filename, frequencies, complex_numbers,
                fitted_complex_numbers
            )
    
        except Exception as e:
            print(f"Error occurred for file: {filename}")
            print(f"Error message: {str(e)}")